In [1]:
import pandas as pd
import osmnx as ox
import numpy as np

north = 55.9899
south = 55.8784
east = -3.0418
west = -3.3374

G = ox.graph_from_bbox(north, south, east, west, network_type='drive_service', simplify=True)

In [7]:
def route_length(route):
    length = 0
    for i in range(len(route)-1):
        length += G[route[i]][route[i+1]][0]['length']
    return length/1000

def parsePath(pathStr):
    removeBrackets = pathStr[1:-1]
    split = removeBrackets.split('-')
    return [int(x) for x in split]

def mapit(node_ids_file, df_file):
    node_ids = np.loadtxt( "data/" + node_ids_file, delimiter=',', dtype=int)
    map_index_id = { k:v for (k,v) in zip(list(range(71)),node_ids)}
    df = pd.read_csv('../ACO/results/'+df_file, index_col=False)
    df['bestPath'][0]
    
    parsedPath = parsePath(df['bestPath'][0])
    mappedPath = [map_index_id[x] for x in parsedPath]

    route = ox.shortest_path(G, mappedPath[-1], mappedPath[0], weight='length')
    for i in range(len(mappedPath)-1):
        if ox.shortest_path(G, mappedPath[i], mappedPath[i+1], weight='length') == None:
            print("None")
            print(mappedPath[i], mappedPath[i+1])
        route += ox.shortest_path(G, mappedPath[i], mappedPath[i+1], weight='length')[1:]

    mapit = ox.plot_route_folium(G, route, route_color='red')

    import folium
    x = [G.nodes[map_index_id[i]]['x'] for i in range(len(node_ids))]
    y = [G.nodes[map_index_id[i]]['y'] for i in range(len(node_ids))]
    coords = list(zip(y, x))
    for location in coords:
        folium.Marker(location=location).add_to(mapit)

    print(route_length(route))

    mapit.save('temp_map.html')

    return mapit

In [8]:
mapit('bins_normal_WS:_Food_Waste_Communal_City_Centre_node_ids_71.csv', 'result_bins_normal_WS:_Food_Waste_Communal_City_Centre_nonplanar_71.csv')

27.738021999999994


In [9]:
mapit('bins_normal_WS:_Food_Waste_Communal_City_Centre_node_ids_71.csv', 'result_bins_normal_WS:_Food_Waste_Communal_City_Centre_nonplanar_71.csv')

27.738021999999994


In [66]:
mapit('bins_normal_WS:_Bottle_Bank_-_Clear_node_ids_29.csv', 'result_bins_normal_WS:_Bottle_Bank_-_Clear_nonplanar_29.csv')

71.06645999999985


In [67]:
mapit('bins_normal_WS:_Bottle_Bank_-_Clear_node_ids_29.csv', 'result_bins_normal_WS:_Bottle_Bank_-_Clear_nonplanar_29.csv')

73.44617499999988
